In [22]:
from dataset.sequence import SkeletonClusteredSequence, ClusteredSequence
import pickle as pkl
import mxnet as mx
from tqdm import tqdm
from copy import copy

with open('clustered_sequences.pkl', 'rb') as file:
    sequences = pkl.load(file)

with open('skeleton_sequences.pkl', 'rb') as file:
    skeletons = pkl.load(file)



In [23]:
clustered_skeleton_sequences = []
for sk_seq in skeletons:
    for seq in sequences:
        if sk_seq.name_pattern == seq.name_pattern and sk_seq.start_frame == seq.gt['frame'].min():
            clustered_skeleton_sequences.append(SkeletonClusteredSequence(seq, sk_seq))
            break
len(clustered_skeleton_sequences)

1692

In [24]:
from utils.missing_data import remove_ellipsis
import numpy as np
from utils.normalizers import skeleton_normalizer


class SinglePersonSequenceHandler:
    def __init__(self, seq: SkeletonClusteredSequence):
        self.seq = seq

    def get_person_num(self):
        return len(self.seq.person_features)

    def get_person_ids(self):
        return list(self.seq.person_features.keys())

    def get_single_person_attributes(self, person_id):
        array_index = self.get_person_ids().index(person_id)
        klass = self.seq.cluster_labels[array_index]
        skeleton_seq, confidence_seq, classes_seq, bboxes_seq, scores_seq = self._get_skeleton_seq(person_id)
        return skeleton_seq, confidence_seq, classes_seq, bboxes_seq, scores_seq, klass

    def get_single_person_as_train_record(self, person_id):
        array_index = self.get_person_ids().index(person_id)
        klass = self.seq.cluster_labels[array_index]
        skeleton_seq = []
        for skeleton_iter, person_ids in zip(self.seq.skeleton_seq.skeletons_seq, self.seq.skeleton_seq.person_ids):
            if person_id not in person_ids:
                skeleton_seq.append(...)
                continue
            person_idx = np.where(person_ids == float(person_id))[0]
            skeleton_seq.append(skeleton_normalizer(skeleton_iter[person_idx]))
        filled_skeletons =remove_ellipsis(skeleton_seq)


        return filled_skeletons, klass

    def _get_skeleton_seq(self, person_id):
        skeleton_seq = []
        confidence_seq = []
        classes_seq = []
        bboxes_seq = []
        scores_seq = []
        for skeleton_iter, confidence_iter, classes_iter, bboxes_iter, scores_iter, person_ids in zip(self.seq.skeleton_seq.skeletons_seq,
                                                                             self.seq.skeleton_seq.confidence_seq,
                                                                             self.seq.skeleton_seq.classes_seq,
                                                                             self.seq.skeleton_seq.bboxes_seq,
                                                                             self.seq.skeleton_seq.scores_seq,
                                                                             self.seq.skeleton_seq.person_ids):
            if person_id not in person_ids:
                skeleton_seq.append(...)
                confidence_seq.append(...)
                classes_seq.append(...)
                bboxes_seq.append(...)
                scores_seq.append(...)
                continue
            person_idx = np.where(person_ids == float(person_id))[0]
            skeleton_seq.append(skeleton_iter[person_idx])
            confidence_seq.append(confidence_iter[person_idx])
            classes_seq.append(classes_iter[0][person_idx])
            bboxes_seq.append(bboxes_iter[0][person_idx])
            scores_seq.append(scores_iter[0][person_idx])
        return remove_ellipsis(skeleton_seq), remove_ellipsis(confidence_seq), remove_ellipsis(
            classes_seq), remove_ellipsis(bboxes_seq), remove_ellipsis(scores_seq)


handler = SinglePersonSequenceHandler(clustered_skeleton_sequences[0])
# skeleton, confidence, classes, bboxes, scores, klass = handler.get_single_person_attributes(4)
skeleton, klass = handler.get_single_person_as_train_record(4)


In [25]:
from tqdm import tqdm
import gluoncv
import cv2
#
# seq = sequences[0]
# gt = seq.gt
# start_frame = gt['frame'].min()
# end_frame = gt['frame'].max()
# is_in_frames = ((gt['frame'] >= start_frame) & (gt['frame'] <= end_frame))
#
# for frame in tqdm(range(start_frame, end_frame + 1)):
#     img = cv2.imread(seq.name_pattern.format(frame=frame))
#     ax = gluoncv.utils.viz.cv_plot_keypoints(img,
#                                             # skeleton_normalizer(skeleton[frame - start_frame]),
#                                              skeleton[frame - start_frame],
#                                              confidence[frame - start_frame],
#                                              np.array([classes[frame - start_frame]]), np.array([bboxes[frame - start_frame]]),
#                                              np.array([scores[frame - start_frame]]),
#                                              box_thresh=0.5, keypoint_thresh=0.0)
#     # labels=det_labels, colors=colors)
#     cv2.imshow('frame', ax)
#     cv2.waitKey(0)
# cv2.destroyAllWindows()
# cv2.waitKey(1)


In [26]:
X = []
y = []
for seq in tqdm(clustered_skeleton_sequences):
    handler = SinglePersonSequenceHandler(seq)
    for person_id in handler.get_person_ids():
        skeleton, klass = handler.get_single_person_as_train_record(person_id)
        X.append(np.squeeze(np.array(skeleton)))
        y.append(klass)
print(len(X           ))
X[0].shape, y[0]

100%|██████████| 1692/1692 [00:18<00:00, 90.27it/s]

19632


((32, 17, 2), 6)

In [28]:
with open('single_X.pkl', 'wb') as file:
    pkl.dump(                           X, file)

with open('single_y.pkl', 'wb') as file:
    pkl.dump(y, file)